In [ ]:
import numpy as np
import pandas as pd 
import os
from collections import defaultdict
from PIL import Image
import random
import matplotlib.pyplot as plt
import cv2

In [ ]:
def get_unique_sizes(directory):
    size_counts = defaultdict(int)
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    with Image.open(os.path.join(root, file)) as img:
                        size = img.size
                        size_counts[size] += 1
                except Exception as e:
                    print(f"Error {file}: {e}")
    return size_counts

base_path = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"

splits = {
    "train": [
        os.path.join(base_path, "train_images/authentic"),
        os.path.join(base_path, "train_images/forged"),
    ],
    "supplemental": [
        os.path.join(base_path, "supplemental_images"),
    ],
    "test": [
        os.path.join(base_path, "test_images"),
    ],
}

total_summary = {}
train_auth_count = 0
train_forged_count = 0

for split_name, dirs in splits.items():
    agg_sizes = defaultdict(int)

    for d in dirs:
        sizes = get_unique_sizes(d)

        # train authentic/forged counts
        if split_name == "train":
            if d.endswith("authentic"):
                train_auth_count += sum(sizes.values())
            elif d.endswith("forged"):
                train_forged_count += sum(sizes.values())

        for size, count in sizes.items():
            agg_sizes[size] += count

    total_images = sum(agg_sizes.values())
    unique_sizes = len(agg_sizes)
    total_summary[split_name] = total_images

    print(f"\n=== {split_name.upper()} ===")
    print(f"Total images: {total_images}")
    print(f"Unique sizes: {unique_sizes}")

    top5 = sorted(agg_sizes.items(), key=lambda x: x[1], reverse=True)[:5]
    print("Top 5 sizes (width, height) -> count:")
    for (w, h), count in top5:
        print(f"  ({w}, {h}) -> {count}")

# explicit train breakdown
print("\n=== TRAIN BREAKDOWN ===")
print(f"Authentic: {train_auth_count}")
print(f"Forged:    {train_forged_count}")

print("\n=== SUMMARY TOTALS ===")
for split_name, total_images in total_summary.items():
    print(f"{split_name.capitalize()}: {total_images} images")

In [ ]:
def count_npy(directory):
    count = 0
    for root, _, files in os.walk(directory):
        for f in files:
            if f.lower().endswith(".npy"):
                count += 1
    return count

base_path = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"

train_masks_dir = os.path.join(base_path, "train_masks")
supp_masks_dir  = os.path.join(base_path, "supplemental_masks")

train_masks_count = count_npy(train_masks_dir)
supp_masks_count  = count_npy(supp_masks_dir)

print("\n=== MASK COUNTS ===")
print(f"Train masks:        {train_masks_count}")
print(f"Supplemental masks: {supp_masks_count}")

In [ ]:
def random_mask_shapes(directory, k=5):
    # collect .npy files
    mask_files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.lower().endswith(".npy")
    ]
    if not mask_files:
        print(f"No masks found in {directory}")
        return

    # pick up to k random masks
    sample = random.sample(mask_files, min(k, len(mask_files)))

    print(f"\nShapes from {directory}:")
    for path in sample:
        arr = np.load(path)
        print(f"{os.path.basename(path)} -> {arr.shape}")

random_mask_shapes(train_masks_dir)
random_mask_shapes(supp_masks_dir)

In [ ]:
def check_masks_channel_first(directory, max_channels=10):
    mask_files = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.lower().endswith(".npy")
    ]

    non_channel_first = []

    for path in mask_files:
        arr = np.load(path, mmap_mode="r")
        shape = arr.shape

        is_channel_first = (len(shape) > 0 and shape[0] <= max_channels)

        if not is_channel_first:
            non_channel_first.append((path, shape))

    if non_channel_first:
        print(f"[WARNING] {len(non_channel_first)} masks in {directory} are NOT channel-first:")
        for p, s in non_channel_first[:10]:
            print(f"  {os.path.basename(p)} -> {s}")
        return True

    print(f"All masks in {directory} appear channel-first.")
    return False


train_bad = check_masks_channel_first(train_masks_dir)
supp_bad  = check_masks_channel_first(supp_masks_dir)

if train_bad or supp_bad:
    print("\n>> Some masks are NOT channel-first.")
else:
    print("\n>> All masks appear channel-first.")


In [ ]:
def summarize_mask_regions(*mask_dirs):
    total_files = 0
    total_regions = 0
    images_with_multiple = 0
    max_regions = 0

    for mask_dir in mask_dirs:
        for fname in os.listdir(mask_dir):
            if not fname.lower().endswith(".npy"):
                continue

            path = os.path.join(mask_dir, fname)
            arr = np.load(path)

            if arr.ndim == 2:
                n_regions = 1
            elif arr.ndim == 3:
                n_regions = arr.shape[0]
            else:
                print(f"Skipping {fname} (in {mask_dir}): unexpected shape {arr.shape}")
                continue

            total_files += 1
            total_regions += n_regions

            if n_regions > 1:
                images_with_multiple += 1

            if n_regions > max_regions:
                max_regions = n_regions

    print("\n=== MASK REGION SUMMARY (train + supplemental) ===")
    print(f"Mask files (images with masks): {total_files}")
    print(f"Total individual regions:       {total_regions}")
    print(f"Images with ≥2 regions:         {images_with_multiple}")
    print(f"Max regions in one image:       {max_regions}")


summarize_mask_regions(train_masks_dir, supp_masks_dir)

In [ ]:
# collect region counts for each mask file (train + supplemental)
region_counts = []

for mask_dir in (train_masks_dir, supp_masks_dir):
    for fname in os.listdir(mask_dir):
        if fname.lower().endswith(".npy"):
            arr = np.load(os.path.join(mask_dir, fname))
            region_counts.append(1 if arr.ndim == 2 else arr.shape[0])

plt.hist(region_counts, bins=range(1, max(region_counts)+2), align='left', rwidth=0.8)
plt.xlabel("Number of forgery regions")
plt.ylabel("Count of images")
plt.title("Histogram of Forgery Regions per Mask (Train + Supplemental)")
plt.xticks(range(1, max(region_counts)+1))
plt.show()


In [ ]:
forgery_sizes = []          # pixel count per region
forgery_percentages = []    # percentage of image area per region

for mask_dir in (train_masks_dir, supp_masks_dir):
    for fname in os.listdir(mask_dir):
        if fname.lower().endswith(".npy"):
            arr = np.load(os.path.join(mask_dir, fname))

            if arr.ndim == 2:
                arr = arr[np.newaxis, :, :]

            C, H, W = arr.shape
            img_area = H * W

            for c in range(C):
                region_pixels = arr[c].sum()
                forgery_sizes.append(region_pixels)
                forgery_percentages.append(region_pixels / img_area * 100.0)

forgery_sizes = np.array(forgery_sizes)
forgery_percentages = np.array(forgery_percentages)

print("\n=== FORGERY SIZE SUMMARY (pixels per region) ===")
print(f"Total regions:      {len(forgery_sizes)}")
print(f"Mean size:          {forgery_sizes.mean():.2f}")
print(f"Median size:        {np.median(forgery_sizes):.2f}")
print(f"Min size:           {forgery_sizes.min()}")
print(f"Max size:           {forgery_sizes.max()}")
print(f"Std deviation:      {forgery_sizes.std():.2f}")

print("\n=== FORGERY SIZE AS % OF IMAGE AREA ===")
print(f"Mean %:             {forgery_percentages.mean():.4f}")
print(f"Median %:           {np.median(forgery_percentages):.4f}")
print(f"Min %:              {forgery_percentages.min():.6f}")
print(f"Max %:              {forgery_percentages.max():.4f}")
print(f"Std deviation %:    {forgery_percentages.std():.4f}")

plt.hist(forgery_percentages, bins=50)
plt.xlabel("Forgery region size (% of image area)")
plt.ylabel("Frequency")
plt.title("Distribution of Forgery Region Sizes (Percent of Image Area)")
plt.show()


In [ ]:
def analyze_region_filters(max_width=5, max_height=5, area_thresholds=(20, 25, 40, 50)):
    total_regions = 0
    filtered_width = 0
    filtered_height = 0
    filtered_either = 0
    pixel_areas = []

    for mask_dir in (train_masks_dir, supp_masks_dir):
        for fname in os.listdir(mask_dir):
            if not fname.lower().endswith(".npy"):
                continue

            arr = np.load(os.path.join(mask_dir, fname))

            if arr.ndim == 2:
                arr = arr[np.newaxis, :, :]

            C, H, W = arr.shape

            for c in range(C):
                region = arr[c]

                if not np.any(region):
                    continue

                ys, xs = np.where(region > 0)
                h = ys.max() - ys.min() + 1
                w = xs.max() - xs.min() + 1

                total_regions += 1

                flag_w = w <= max_width
                flag_h = h <= max_height

                if flag_w:
                    filtered_width += 1
                if flag_h:
                    filtered_height += 1
                if flag_w or flag_h:
                    filtered_either += 1

                region_pixels = region.sum()
                pixel_areas.append(region_pixels)

    print("\n=== SMALL REGION FILTER (by width / height) ===")
    print(f"Total regions considered:            {total_regions}")
    print(f"Filtered due to width <= {max_width}:   {filtered_width}")
    print(f"Filtered due to height <= {max_height}:  {filtered_height}")
    print(f"Filtered due to width OR height:     {filtered_either}")

    pixel_areas = np.array(pixel_areas)

    print("\n=== SMALL REGION FILTER (by pixel area) ===")
    if len(pixel_areas) == 0:
        print("No regions found.")
        return

    for thr in area_thresholds:
        n_filtered = (pixel_areas <= thr).sum()
        pct = n_filtered / len(pixel_areas) * 100.0
        print(f"Area <= {thr:4d}: filtered regions = {n_filtered} ({pct:.2f}% of regions)")


analyze_region_filters()

In [ ]:
from scipy.ndimage import label, sobel, binary_dilation
from skimage.morphology import convex_hull_image

region_areas = []
region_components = []
region_aspect = []
region_elongation = []
region_convexity = []
region_entropy = []
region_edge_density = []
region_centroid_y = []
region_centroid_x = []
region_border_dist = []
regions_per_img = []
area_per_img = []

forged_dir = os.path.join(base_path, "train_images", "forged")
img_index = {}

for img_name in os.listdir(forged_dir):
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_stem = os.path.splitext(img_name)[0]
        img_index[img_stem] = os.path.join(forged_dir, img_name)

for mask_name in os.listdir(train_masks_dir):
    if not mask_name.lower().endswith(".npy"):
        continue

    mask_id = os.path.splitext(mask_name)[0]
    img_path = img_index.get(mask_id)
    if img_path is None:
        continue

    mask_arr = np.load(os.path.join(train_masks_dir, mask_name))
    if mask_arr.ndim == 2:
        mask_arr = mask_arr[np.newaxis, :, :]
    ch, ht, wd = mask_arr.shape

    img_gray = np.array(Image.open(img_path).convert("L"), dtype=np.float32)

    img_region_count = 0
    img_region_area = 0

    gx = sobel(img_gray, axis=1)
    gy = sobel(img_gray, axis=0)
    edge_mag = np.hypot(gx, gy)

    for ci in range(ch):
        region = mask_arr[ci] > 0
        pix = region.sum()
        if pix == 0:
            continue

        img_region_count += 1
        img_region_area += pix
        region_areas.append(pix)

        # connected components
        _, n_comp = label(region)
        region_components.append(n_comp)

        ys, xs = np.where(region)
        y0, y1 = ys.min(), ys.max()
        x0, x1 = xs.min(), xs.max()
        h_box = y1 - y0 + 1
        w_box = x1 - x0 + 1
        region_aspect.append(h_box / w_box if w_box > 0 else 0.0)

        coords = np.stack([xs, ys], axis=1).astype(np.float32)
        coords -= coords.mean(axis=0, keepdims=True)
        cov = coords.T @ coords / coords.shape[0]
        eigvals, _ = np.linalg.eig(cov)
        eigvals = np.sort(np.real(eigvals))
        if eigvals[0] > 1e-6:
            region_elongation.append(float(np.sqrt(eigvals[-1] / eigvals[0])))
        else:
            region_elongation.append(0.0)

        hull = convex_hull_image(region)
        hull_area = hull.sum()
        region_convexity.append(pix / hull_area if hull_area > 0 else 0.0)

        cy = ys.mean() / ht
        cx = xs.mean() / wd
        region_centroid_y.append(cy)
        region_centroid_x.append(cx)
        region_border_dist.append(min(cy, 1 - cy, cx, 1 - cx))

        vals = img_gray[region]
        hist, _ = np.histogram(vals, bins=32, range=(0, 255), density=True)
        hist = hist[hist > 0]
        region_entropy.append(float(-(hist * np.log2(hist)).sum()))

        region_edge_density.append(float(edge_mag[region].mean()))

    if img_region_count > 0:
        regions_per_img.append(img_region_count)
        area_per_img.append(img_region_area)

region_areas = np.array(region_areas)
region_components = np.array(region_components)
region_aspect = np.array(region_aspect)
region_elongation = np.array(region_elongation)
region_convexity = np.array(region_convexity)
region_entropy = np.array(region_entropy)
region_edge_density = np.array(region_edge_density)
region_centroid_y = np.array(region_centroid_y)
region_centroid_x = np.array(region_centroid_x)
region_border_dist = np.array(region_border_dist)
regions_per_img = np.array(regions_per_img)
area_per_img = np.array(area_per_img)

print("\n=== REGION COMPLEXITY ===")
print("Regions:", len(region_areas))
print("Connected components: mean", region_components.mean(), "max", region_components.max())
print("Aspect ratio: mean", region_aspect.mean())
print("Elongation: mean", region_elongation.mean())
print("Convexity: mean", region_convexity.mean())
print("Entropy: mean", region_entropy.mean())
print("Edge density: mean", region_edge_density.mean())

print("\n=== SPATIAL DISTRIBUTION ===")
print("Centroid Y (0=top,1=bottom): mean", region_centroid_y.mean())
print("Centroid X (0=left,1=right): mean", region_centroid_x.mean())
print("Border distance (normalized): mean", region_border_dist.mean(), "min", region_border_dist.min())

print("\n=== REGION SCALE PATTERNS ===")
log_area = np.log10(region_areas + 1)
print("Log10(area): mean", log_area.mean(), "std", log_area.std())
print("Regions per image: mean", regions_per_img.mean(), "max", regions_per_img.max())
corr = np.corrcoef(regions_per_img, area_per_img)[0, 1]
print("Corr(regions per image, total forged area):", corr)

# This helps decide whether to emphasize:

# fine-grained detail (small forgeries)

# global context (large forgeries)

In [ ]:
from skimage.metrics import structural_similarity as ssim
from scipy.ndimage import binary_dilation, binary_erosion

def load_forged_gray(mask_fname):
    stem = os.path.splitext(mask_fname)[0]
    for ext in [".png", ".jpg", ".jpeg"]:
        p = os.path.join(forged_dir, stem + ext)
        if os.path.exists(p):
            return np.array(Image.open(p).convert("L"), dtype=np.float32) / 255.0
    return None

# Texture consistency: border inside vs border outside (SSIM)
def analyze_texture_consistency(n_samples=100):
    mask_files = [f for f in os.listdir(train_masks_dir) if f.lower().endswith(".npy")]
    if not mask_files:
        print("No mask files found.")
        return

    chosen = random.sample(mask_files, min(n_samples, len(mask_files)))
    ssim_scores = []

    for fname in chosen:
        m_arr = np.load(os.path.join(train_masks_dir, fname))
        if m_arr.ndim == 2:
            m_arr = m_arr[np.newaxis, :, :]

        img = load_forged_gray(fname)
        if img is None:
            continue

        for c in range(m_arr.shape[0]):
            m = m_arr[c] > 0.5
            if m.sum() < 50:
                continue

            dil = binary_dilation(m, iterations=2)
            ero = binary_erosion(m, iterations=2)

            border_in = m & ~ero
            border_out = dil & ~m

            if border_in.sum() < 20 or border_out.sum() < 20:
                continue

            ys, xs = np.where(dil)
            y0, y1 = ys.min(), ys.max() + 1
            x0, x1 = xs.min(), xs.max() + 1

            img_roi = img[y0:y1, x0:x1]
            bi_roi = border_in[y0:y1, x0:x1]
            bo_roi = border_out[y0:y1, x0:x1]

            inside_img = img_roi * bi_roi
            outside_img = img_roi * bo_roi

            try:
                score = ssim(inside_img, outside_img, data_range=1.0)
                ssim_scores.append(score)
            except Exception:
                continue

    if not ssim_scores:
        print("No valid border pairs for SSIM.")
        return

    ssim_scores = np.array(ssim_scores)
    print("\n=== TEXTURE CONSISTENCY (border inside vs outside) ===")
    print(f"Samples used: {len(ssim_scores)}")
    print(f"Mean SSIM:    {ssim_scores.mean():.4f}")
    print(f"Median SSIM:  {np.median(ssim_scores):.4f}")
    print(f"Min / Max:    {ssim_scores.min():.4f} / {ssim_scores.max():.4f}")

# Copy–move characteristics via global autocorrelation (dominant displacement)
def analyze_copy_move_displacements(n_samples=10):
    mask_files = [f for f in os.listdir(train_masks_dir) if f.lower().endswith(".npy")]
    if not mask_files:
        print("No mask files found.")
        return

    chosen = random.sample(mask_files, min(n_samples, len(mask_files)))
    displacements = []

    print("\n=== COPY–MOVE DISPLACEMENT ESTIMATES (autocorrelation peaks) ===")
    for fname in chosen:
        img = load_forged_gray(fname)
        if img is None:
            continue

        # zero-mean for autocorrelation
        im = img - img.mean()
        fft = np.fft.fft2(im)
        ac = np.fft.ifft2(np.abs(fft) ** 2).real
        ac_shift = np.fft.fftshift(ac)

        # center is zero displacement
        h, w = ac_shift.shape
        cy, cx = h // 2, w // 2

        ac_flat = ac_shift.copy()
        ac_flat[cy, cx] = -np.inf  # ignore zero-lag peak

        idx = np.argmax(ac_flat)
        dy, dx = np.unravel_index(idx, ac_shift.shape)
        dy -= cy
        dx -= cx

        displacements.append((dy, dx))
        print(f"{fname}: strongest non-zero displacement ~ (dy={dy}, dx={dx})")

    if displacements:
        dy_vals = np.array([d[0] for d in displacements])
        dx_vals = np.array([d[1] for d in displacements])
        print("\nSummary of dominant displacements (pixels):")
        print(f"Mean dy, dx: ({dy_vals.mean():.2f}, {dx_vals.mean():.2f})")
        print(f"Median |dy|, |dx|: ({np.median(np.abs(dy_vals)):.2f}, {np.median(np.abs(dx_vals)):.2f})")

# run both analyses
analyze_texture_consistency()
analyze_copy_move_displacements()

# A feature that searches for duplicated features (ORB/SIFT keypoints)

# A model architecture focusing on patch-matching (like PatchCore or Swin with windowed attention)

In [ ]:
def analyze_copy_move_displacements_orb_local(
    n_samples=100,
    max_kp=5000,
    min_disp=3,
    min_matches=10
):
    mask_files = [f for f in os.listdir(train_masks_dir) if f.lower().endswith(".npy")]
    if not mask_files:
        print("No mask files found.")
        return

    chosen = random.sample(mask_files, min(n_samples, len(mask_files)))
    all_dy, all_dx = [], []

    print("\n=== COPY–MOVE DISPLACEMENT ESTIMATES (ORB keypoint matches, local) ===")
    orb = cv2.ORB_create(max_kp)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    for fname in chosen:
        # load mask
        m_arr = np.load(os.path.join(train_masks_dir, fname))
        if m_arr.ndim == 2:
            m = m_arr > 0.5
        else:
            m = (m_arr > 0.5).any(axis=0)  # union over channels

        # load forged image
        img_f = load_forged_gray(fname)
        if img_f is None:
            continue

        img_u8 = (img_f * 255).astype(np.uint8)

        # detect ORB keypoints
        kps, desc = orb.detectAndCompute(img_u8, None)
        if desc is None or len(kps) < 2:
            continue

        # split keypoints into inside-mask (forged) and outside-mask (background)
        inside_idx, outside_idx = [], []
        h, w = m.shape
        for i, kp in enumerate(kps):
            x, y = int(round(kp.pt[0])), int(round(kp.pt[1]))
            if 0 <= x < w and 0 <= y < h:
                if m[y, x]:
                    inside_idx.append(i)
                else:
                    outside_idx.append(i)

        if len(inside_idx) == 0 or len(outside_idx) == 0:
            continue

        desc_in  = desc[inside_idx]
        desc_out = desc[outside_idx]
        kps_in   = [kps[i] for i in inside_idx]
        kps_out  = [kps[i] for i in outside_idx]

        if len(desc_in) == 0 or len(desc_out) == 0:
            continue

        # knn matching: forged region -> background
        knn_matches = bf.knnMatch(desc_in, desc_out, k=2)
        disps = []

        for mlist in knn_matches:
            if len(mlist) < 2:
                continue
            m1, m2 = mlist
            # Lowe ratio test
            if m1.distance >= 0.75 * m2.distance:
                continue

            q_idx = m1.queryIdx
            t_idx = m1.trainIdx
            
            p1 = kps_in[q_idx].pt   # inside (forged)
            p2 = kps_out[t_idx].pt  # outside (source/background)

            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            if abs(dx) + abs(dy) < min_disp:
                continue  # ignore tiny shifts

            disps.append((dy, dx))

        if len(disps) < min_matches:
            continue

        dy_arr = np.array([d[0] for d in disps])
        dx_arr = np.array([d[1] for d in disps])

        dom_dy = np.median(dy_arr)
        dom_dx = np.median(dx_arr)

        print(f"{fname}: ~ (dy={dom_dy:.1f}, dx={dom_dx:.1f}) "
              f"from {len(disps)} matches")

        all_dy.extend(dy_arr)
        all_dx.extend(dx_arr)

    if all_dy:
        all_dy = np.array(all_dy)
        all_dx = np.array(all_dx)
        print("\nSummary over all images (pixels):")
        print(f"Mean dy, dx: ({all_dy.mean():.2f}, {all_dx.mean():.2f})")
        print(f"Median dy, dx: ({np.median(all_dy):.2f}, {np.median(all_dx):.2f})")
        print(f"Median |dy|, |dx|: ({np.median(np.abs(all_dy)):.2f}, "
              f"{np.median(np.abs(all_dx)):.2f})")
    else:
        print("No non-trivial displacements found (after mask-aware matching).")

# run it
analyze_copy_move_displacements_orb_local()

In [ ]:
def _mask_perimeter(mask2d):
    m = mask2d.astype(bool)
    if m.sum() == 0:
        return 0
    p = np.pad(m, 1, constant_values=False)
    core = p[1:-1, 1:-1]
    perim = 0
    perim += np.logical_and(core, ~p[1:-1, 2:]).sum()   # right
    perim += np.logical_and(core, ~p[1:-1, 0:-2]).sum() # left
    perim += np.logical_and(core, ~p[2:, 1:-1]).sum()   # down
    perim += np.logical_and(core, ~p[0:-2, 1:-1]).sum() # up
    return perim

def _fractal_dim(mask2d):
    m = mask2d.astype(bool)
    h, w = m.shape
    sizes = [1, 2, 4, 8, 16, 32]
    eps = []
    counts = []
    for s in sizes:
        if s > min(h, w):
            continue
        hh = (h // s) * s
        ww = (w // s) * s
        if hh == 0 or ww == 0:
            continue
        mm = m[:hh, :ww]
        mm = mm.reshape(hh // s, s, ww // s, s)
        red = mm.max(axis=(1, 3))
        n = red.sum()
        if n > 0:
            eps.append(1.0 / s)
            counts.append(n)
    if len(counts) < 2:
        return np.nan
    eps = np.array(eps, dtype=float)
    counts = np.array(counts, dtype=float)
    coef = np.polyfit(np.log(1.0 / eps), np.log(counts), 1)
    return coef[0]

def analyze_forgery_nature(mask_dir, image_dir, max_images=None):
    perim_area_ratios = []
    jaggedness_vals = []
    fractal_vals = []
    bg_vars = []
    bg_edges = []

    processed_images = 0

    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(".npy"):
            continue

        base = os.path.splitext(fname)[0]
        img_path = None
        for ext in [".png", ".jpg", ".jpeg"]:
            candidate = os.path.join(image_dir, base + ext)
            if os.path.exists(candidate):
                img_path = candidate
                break
        if img_path is None:
            continue

        arr = np.load(os.path.join(mask_dir, fname))
        if arr.ndim == 2:
            arr = arr[np.newaxis, :, :]
        C, H, W = arr.shape

        img = Image.open(img_path).convert("L")
        img_arr = np.array(img, dtype=np.float32) / 255.0

        for c in range(C):
            m = arr[c].astype(bool)
            area = m.sum()
            if area == 0:
                continue

            # A. Boundary complexity
            perim = _mask_perimeter(m)
            perim_area_ratios.append(perim / area)
            circ_perim = 2.0 * np.sqrt(np.pi * area)
            jaggedness_vals.append(perim / (circ_perim + 1e-8))
            fractal_vals.append(_fractal_dim(m))

            # B. Region interaction (simple background complexity proxy)
            ys, xs = np.where(m)
            y0, y1 = ys.min(), ys.max() + 1
            x0, x1 = xs.min(), xs.max() + 1
            patch = img_arr[y0:y1, x0:x1]
            if patch.size == 0:
                continue

            bg_vars.append(patch.var())
            gx = np.abs(patch[:, 1:] - patch[:, :-1])
            gy = np.abs(patch[1:, :] - patch[:-1, :])
            
            edge_vals = []
            if gx.size > 0:
                edge_vals.append(gx.mean())
            if gy.size > 0:
                edge_vals.append(gy.mean())
            
            if edge_vals:
                bg_edges.append(float(np.mean(edge_vals)))
            else:
                # degenerate 1x1 patch: no edges; treat as 0 complexity
                bg_edges.append(0.0)

        processed_images += 1
        if max_images is not None and processed_images >= max_images:
            break

    perim_area_ratios = np.array(perim_area_ratios, dtype=float)
    jaggedness_vals = np.array(jaggedness_vals, dtype=float)
    fractal_vals = np.array(fractal_vals, dtype=float)
    bg_vars = np.array(bg_vars, dtype=float)
    bg_edges = np.array(bg_edges, dtype=float)

    print("\n=== BOUNDARY COMPLEXITY (per region) ===")
    print(f"Regions analyzed:             {len(perim_area_ratios)}")
    print(f"Perimeter/area mean:          {perim_area_ratios.mean():.4f}")
    print(f"Perimeter/area median:        {np.median(perim_area_ratios):.4f}")
    print(f"Jaggedness mean (circle=1):   {jaggedness_vals.mean():.4f}")
    print(f"Jaggedness median:            {np.median(jaggedness_vals):.4f}")
    print(f"Fractal dimension mean:       {np.nanmean(fractal_vals):.4f}")
    print(f"Fractal dimension median:     {np.nanmedian(fractal_vals):.4f}")

    print("\n=== REGION INTERACTION PROXIES (image patch under mask) ===")
    print(f"Patch variance mean:          {bg_vars.mean():.6f}")
    print(f"Patch variance median:        {np.median(bg_vars):.6f}")
    print(f"Edge magnitude mean:          {bg_edges.mean():.6f}")
    print(f"Edge magnitude median:        {np.median(bg_edges):.6f}")

# forged images dir from your earlier splits
forged_images_dir = splits["train"][1]

analyze_forgery_nature(train_masks_dir, forged_images_dir, max_images=None)

# Certain architectures excel at detecting:

# Smooth blending

# Hard cut-and-paste edges

# Irregular cloning artifacts

# This helps decide whether to incorporate:

# OCR signals

# Structural representations of scientific figures

# Local high-frequency filters